In [ ]:
import warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
warnings.simplefilter('ignore', category=UserWarning)
warnings.filterwarnings('ignore',category=ImportWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

import pandas as pd
import numpy as np
import category_encoders as ce
from scipy.stats import randint
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import ADASYN
from BorutaShap import BorutaShap

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PowerTransformer, OrdinalEncoder, OneHotEncoder
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingRandomSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.utils.fixes import loguniform
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
#from xgboost import XGBClassifier

import joblib
import operator

## Import data from nfl-data-py
##### https://pypi.org/project/nfl-data-py/

In [ ]:
# Import csv file from nfl-data-py
df = pd.read_csv(r'/Users/ttas2/Documents/Python/nfl-machine-learning-models/output_files/nfl_post_processing_run_pass_classification_data.csv')

df.shape

In [ ]:
# Print columns with missing values
print(df.columns[df.isnull().any()].tolist())

In [ ]:
# Convert binary columns to integers
binary_columns = df.columns[df.isin([0,1]).all()].tolist()
df[binary_columns] = df[binary_columns].apply(pd.to_numeric, downcast='integer', errors='coerce', axis=1)

df.sample(2)

In [ ]:
# Target frequency
target_count = df.play_type.value_counts(normalize=True)
target_count

In [ ]:
df['play_type'] = np.where(df['play_type'] == 'pass', 1, 0)

## Train test split data
##### https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# split data into target and feature datasets
X, y = df.loc[:, df.columns != 'play_type'], df['play_type']

initial_features = X.columns.to_list()

# Create train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=67)

print(X_train.shape[1], 'features before feature selection')

## Encode categorical features
##### https://contrib.scikit-learn.org/category_encoders/leaveoneout.html

In [ ]:
# Create list of features for each dtype
categorical_feat = list(X_train.select_dtypes(include='object'))

# One hot encoding of categorical features
# encoder = ce.OneHotEncoder(return_df=True, cols=categorical_feat, use_cat_names=True)

# Encoded column represents mean response over all rows for this category, providing one-column representation while avoiding direct response leakage
encoder = ce.LeaveOneOutEncoder(return_df=True, cols=categorical_feat)

X_train = encoder.fit_transform(X_train, y_train)
X_test = encoder.transform(X_test)

print(X_train.shape[1], 'features after categorical encoding')

## BorutaShap Feature selection
##### https://pypi.org/project/BorutaShap/
##### https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [ ]:
# Specify feature selection model
feature_model = GradientBoostingClassifier(loss='log_loss',
                                           n_estimators=200,
                                           max_depth=50,
                                           max_features=0.5,
                                           max_leaf_nodes=50,
                                           subsample=0.5,
                                           tol=1e-06,
                                           learning_rate=0.04,
                                           min_samples_split=0.05,
                                           criterion='friedman_mse',
                                           random_state=67,
                                           )

In [ ]:
# no model selected default is Random Forest
Feature_Selector = BorutaShap(model=feature_model,
                              importance_measure='shap',
                              classification=True,
                              )

Feature_Selector.fit(X=X_train,
                     y=y_train,
                     n_trials=200,
                     sample=False,
                     random_state=67,
                     )

 99%|█████████▉| 298/300 [20:31:46<03:21, 100.63s/it]

In [ ]:
Feature_Selector.plot(which_features='all', figsize=(18, 8))

In [ ]:
# Drops features that were identified by BorutaShap as not important
features_to_remove = Feature_Selector.features_to_remove

X_train = X_train.drop(columns=features_to_remove)
X_test = X_test.drop(columns=features_to_remove)

# Print the shape of the new datasets
print('Training features:', X_train.shape[1])
print('Testing features:', X_test.shape[1])
print('Remaining features:', X_train.columns.to_list())

## Baseline model
##### https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html

In [ ]:
# Create and fit baseline model to compare performance
baseline_model = DummyClassifier(strategy='most_frequent', random_state=67)
baseline_model.fit(X_train, y_train)

# Calculate model accuracy on test data
y_baseline_pred = baseline_model.predict(X_test)
accuracy_score(y_test, y_baseline_pred)
print(f"Baseline test accuracy: {(round(accuracy_score(y_test,y_baseline_pred),3)*100)}%")

## Model pipeline
##### https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html
##### https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html
##### https://imbalanced-learn.org/stable/references/over_sampling.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html
##### https://xgboost.readthedocs.io/en/stable/parameter.html

In [ ]:
# Create feature type lists for column transform stage of the pipeline
ordinal_features = X_train.columns[X_train.isin([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]).all()].tolist()
categorical_features = list(X_train.select_dtypes(include='object'))
boolean_features = X_train.columns[X_train.isin([0, 1]).all()].tolist()

numeric_features = [x for x in X_train.columns if x not in ordinal_features]
numeric_features = [x for x in numeric_features if x not in categorical_features]
numeric_features = [x for x in numeric_features if x not in boolean_features]

# print('categorical features:', len(categorical_features), ':', categorical_features)
print('ordinal features:', len(ordinal_features), ':', ordinal_features)
print('boolean features:', len(boolean_features), ':', boolean_features)
print('numeric features:', len(numeric_features), ':', numeric_features)

In [ ]:
# Outlier removal
def IQR_Outliers(x, features):

    out_index_list = []
        
    for col in features:
        q1 = np.nanpercentile(x[col], 25.)
        q3 = np.nanpercentile(x[col], 75.)
        
        cut_off = (q3 - q1) * 1.5
        upper, lower = q3 + cut_off, q1 - cut_off
                
        outliers_index = x[col][(x[col] < lower) | (x[col] > upper)].index.tolist()
        out_index_list.extend(outliers_index)
        
    # Remove duplicates
    list(set(out_index_list)).sort()

In [ ]:
# Specify the transformations per data type
num_trans = Pipeline(steps=[('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
                            ('iqr_outlier', IQR_Outliers(X_train, numeric_features)),
                            ('power_trans', PowerTransformer(method='yeo-johnson', copy=False)),
                            ('std_scaler', StandardScaler()),
                           ])

cat_trans = Pipeline(steps=[('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                            ('onehot_encoder', OneHotEncoder(sparse=False, handle_unknown='raise')),
                           ])

ord_trans = Pipeline(steps=[('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                            ('ordinal_encoder', OrdinalEncoder()),
                            ('std_scaler', StandardScaler()),
                           ])
                                
Column_Transform = ColumnTransformer(transformers=[('numeric_transform', num_trans, numeric_features),
                                                  ('categorical_transform', cat_trans, categorical_features),
                                                  ('ordinal_transform', ord_trans, ordinal_features),
                                                  ],
                                    remainder='passthrough',
                                    )

In [ ]:
# Specify number of target classes
n_classes = y_train.nunique()

# Specify HalvingRandomSearchCV halving_parameter (aka factor)
halving_parameter = 2.0

# Specify the HalvingRandomSearchCV minimum/maximum resources
max_resource = 2500
resource_divisor = 2.0
min_resource = int(round((max_resource / resource_divisor), 0))

In [ ]:
def random_search():
    
    pipeline1 = Pipeline([
    ('col', Column_Transform),
    ('smpl', ADASYN(n_neighbors=n_classes, sampling_strategy='not majority', random_state=67)),
    ('clf', RandomForestClassifier()),
    ])
    
    pipeline2 = Pipeline([
    ('col', Column_Transform),
    ('smpl', ADASYN(n_neighbors=n_classes, sampling_strategy='not majority', random_state=67)),
    ('clf', ExtraTreesClassifier()),
    ])
    
    pipeline3 = Pipeline([
    ('col', Column_Transform),
    ('smpl', ADASYN(n_neighbors=n_classes, sampling_strategy='not majority', random_state=67)),
    ('clf', GradientBoostingClassifier()),
    ])

    # RandomForestClassifier
    parameters1 = {
    'clf__criterion': ['gini','entropy'],
    'clf__max_features': loguniform(0.10, 0.75),
    'clf__max_depth': randint(5, 40),
    'clf__min_samples_split': loguniform(0.10, 0.60),
    'clf__min_samples_leaf': loguniform(0.10, 0.60),
    'clf__min_impurity_decrease': loguniform(1e-09, 1e-03),
    'clf__min_weight_fraction_leaf':  loguniform(0.15, 0.70),
    'clf__ccp_alpha':  loguniform(1e-08, 1e-03),
    'clf__bootstrap': [True, False],
    'clf__oob_score': [False],
    'clf__warm_start': [True, False],
    'clf__n_jobs': [6],
    'clf__random_state': [67],
    }
    
    # ExtraTreesClassifier
    parameters2 = {
    'clf__criterion': ['gini'],
    'clf__max_depth': randint(10, 50),
    'clf__max_features': loguniform(0.20, 0.95),
    'clf__max_leaf_nodes': [None],
    'clf__max_samples': [None], #loguniform(0.35, 0.75),
    'clf__min_samples_split': loguniform(0.10, 0.50),
    'clf__min_samples_leaf': loguniform(0.10, 0.50),
    'clf__min_weight_fraction_leaf': loguniform(0.10, 0.60),
    'clf__min_impurity_decrease': loguniform(1e-09, 1e-03),
    'clf__ccp_alpha': loguniform(1e-08, 1e-02),
    'clf__bootstrap': [True, False],
    'clf__oob_score': [False],
    'clf__warm_start': [True, False],
    'clf__n_jobs': [6],
    'clf__random_state': [67],
    }
            
    # GradientBoostingClassifier
    parameters3 = {
    'clf__loss': ['log_loss'],
    'clf__max_features': loguniform(0.10, 0.50),
    'clf__learning_rate': loguniform(1e-07, 1e-02),
    'clf__ccp_alpha': loguniform(1e-07, 1e-02),
    'clf__max_depth': randint(10, 40),
    'clf__max_leaf_nodes': randint(2, 20),
    'clf__min_samples_split': loguniform(0.25, 0.60),
    'clf__min_impurity_decrease': loguniform(1e-08, 1e-02),
    'clf__min_samples_leaf': loguniform(0.25, 0.50),
    'clf__n_iter_no_change': [150, 200, None],
    'clf__tol': loguniform(1e-08, 1e-04),
    'clf__validation_fraction': loguniform(0.05, 0.15),
    'clf__warm_start': [False],
    'clf__random_state': [67],
    }

    pars = [parameters1, parameters2, parameters3]
    pips = [pipeline1, pipeline2, pipeline3]

    result = []
    
    for i in range(len(pars)):
        
        rs = HalvingRandomSearchCV(pips[i],
                                   pars[i],
                                   factor=halving_parameter,
                                   resource='clf__n_estimators',
                                   n_candidates='exhaust',
                                   min_resources=min_resource,
                                   max_resources=max_resource,
                                   scoring='balanced_accuracy',
                                   aggressive_elimination=False,
                                   return_train_score=True,
                                   refit=True,
                                   cv=5,
                                   n_jobs=6,
                                   verbose=1,
                                   random_state=67,
                                   error_score='raise',
                                  )

        # Fit models on training data
        rs = rs.fit(X_train, y_train)
        
        # Apply models to test data to determine model performance
        y_pred = rs.predict(X_test)
        y_pred_prob = rs.predict_proba(X_test)[:, 1]

        print(" ")
        print(" ")
        
        # storing model results
        result.append({
        'grid': rs,
        'cv results': rs.cv_results_,
        'best train balanced accuracy score': rs.best_score_,
        'best params': rs.best_params_, 
        'best estimator': rs.best_estimator_,
        'feature importance': rs.best_estimator_.named_steps['clf'].feature_importances_,
        'best test balanced accuracy score': balanced_accuracy_score(y_test, y_pred),
        'best test roc auc score': roc_auc_score(y_test, y_pred_prob),
        #'test classification report': classification_report(y_test, y_pred, target_names=['pass','run']),
        'cv': rs.cv,
        'model #': i + 1
        })

    # sorting results by best test score
    result = sorted(result, key=operator.itemgetter('best test balanced accuracy score'), reverse=True)
    
    print('Best Models:')
    print(' ')
    for element in result:
        if element['model #']==1:
            print('Random Forest classifier: ')
        elif element['model #']==2:
            print('ExtraTrees classifier: ')
        elif element['model #']==3:
            print('GradientBoosting classifier: ')
        else:
            print('Other: ')  
        print('Parameters:  ' + str(element['best params']))
        print('Train balanced accuracy score:  ' + str(element['best train balanced accuracy score']))
        print('Test balanced accuracy score:   ' + str(element['best test balanced accuracy score']))
        print('Test roc auc score:   ' + str(element['best test roc auc score']))
        print('-----------------------')
        print(' ')
        print(' ')

    # Save best model as pickle file
    joblib.dump(rs.best_params_, r'/Users/ttas2/Documents/Python/nfl-machine-learning-models/output_files/run_pass_classifier_results.pkl', compress=1)

In [ ]:
%%capture --no-stdout --no-display
random_search()

In [ ]:
# Load best model
best_model = joblib.load(r'/Users/ttas2/Documents/Python/nfl-machine-learning-models/output_files/run_pass_classifier_results.pkl')
best_model